# [Integration of lakeFS with Airflow](https://docs.lakefs.io/integrations/airflow.html)

## Use Case: Isolating Airflow job run and atomic promotion to production

## Import Python packages

In [ ]:
import lakefs_client

## Working with the lakeFS Python client API

###### Note: To learn more about lakeFS Python integration visit https://docs.lakefs.io/integrations/python.html

In [ ]:
%xmode Minimal
from lakefs_client import models
from lakefs_client.client import LakeFSClient

# lakeFS credentials and endpoint
configuration = lakefs_client.Configuration()
configuration.username = lakefsAccessKey
configuration.password = lakefsSecretKey
configuration.host = lakefsEndPoint

client = LakeFSClient(configuration)

## Verify lakeFS credentials by getting lakeFS version

In [ ]:
print("Verifying lakeFS credentials")
client.config.get_lake_fs_version()
print("lakeFS credentials verified")

## Start Airflow

In [ ]:
print("Starting Airflow")

In [ ]:
! pkill airflow
! pkill airflow
! pkill airflow

In [ ]:
%%script bash --bg --out script_out --err script_error
airflow standalone

In [ ]:
# Wait for Airflow to start
! sleep 20

In [ ]:
%%script bash --bg --out script_out --err script_error
airflow scheduler

In [ ]:
print("Airflow Started")

## Create Airflow connections for lakeFS

In [ ]:
! airflow connections delete conn_lakefs
lakeFSConnectionCommand = 'airflow connections add conn_lakefs --conn-type=http --conn-host=' + lakefsEndPoint + ' --conn-extra=\'{"access_key_id":"' + lakefsAccessKey + '","secret_access_key":"' + lakefsSecretKey + '"}\''
! $lakeFSConnectionCommand

## Set Airflow variables which are used by the demo workflow

In [ ]:
! airflow variables set lakefsAccessKey $lakefsAccessKey
! airflow variables set lakefsSecretKey $lakefsSecretKey
! airflow variables set lakefsEndPoint $lakefsEndPoint
! airflow variables set repo $repo
! airflow variables set sourceBranch $sourceBranch
! airflow variables set newBranch $newBranch
! airflow variables set conn_lakefs 'conn_lakefs'

## Copy DAG programs to Airflow DAGs directory and sync to Airflow database

In [ ]:
! cp ./airflow/Existing_DAG/lakefs_tutorial_taskflow_api_etl.py ./airflow/dags
! cp ./airflow/Existing_DAG/lakefs_wrapper_dag.py ./airflow/dags

from airflow.models import DagBag
dagbag = DagBag(include_examples=False)
dagbag.sync_to_db()